# Google Analytics Data Cleaning & Visualization

## Project Overview

This notebook demonstrates professional-grade data cleaning, transformation, and visualization techniques using real Google Analytics data. It showcases a complete data wrangling pipeline that transforms raw 150,000+ user session events into actionable business intelligence.

### Key Objectives:
- Clean and validate Google Analytics data
- Remove duplicates and handle missing values
- Filter bot traffic
- Normalize timezones and standardize data
- Generate comprehensive data quality reports
- Visualize insights and patterns


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 1. Data Loading

Load the raw Google Analytics export data.


In [ ]:
# Load the data cleaning class
from data_cleaning_analysis import DataCleaningAnalysis

# Initialize the data cleaning pipeline
cleaner = DataCleaningAnalysis(data_path='data/raw/google_analytics_export.csv')

# Load the data
df = cleaner.load_data()
df.head()


## 2. Initial Data Assessment

Assess the quality and structure of the raw data.


In [ ]:
# Perform initial assessment
cleaner.initial_assessment()

# Display basic statistics
print("\n=== Data Summary ===")
print(f"Total Records: {len(df):,}")
print(f"Total Columns: {len(df.columns)}")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


## 3. Data Cleaning Pipeline

Execute the complete data cleaning pipeline:
1. Remove duplicates
2. Handle missing values
3. Filter bot traffic
4. Normalize timezones
5. Standardize columns


In [ ]:
# Execute the complete cleaning pipeline
cleaner.execute_pipeline()


## 4. Data Quality Report

Review the comprehensive data quality report.


In [ ]:
# Load and display the quality report
quality_report = pd.read_csv('data/processed/data_quality_report.csv')
print("=== Data Quality Report ===")
print(quality_report.to_string(index=False))

# Display summary statistics
print("\n=== Quality Summary ===")
print(f"Total Checks: {len(quality_report)}")
print(f"Passed: {len(quality_report[quality_report['status'] == 'PASS'])}")
print(f"Warnings: {len(quality_report[quality_report['status'] == 'WARNING'])}")
print(f"Failed: {len(quality_report[quality_report['status'] == 'FAIL'])}")


## 5. Data Visualization

Create visualizations to explore the cleaned data.


In [ ]:
# Load cleaned data for visualization
cleaned_df = pd.read_csv('data/processed/cleaned_data.csv')

# Display first few rows
print("=== Cleaned Data Sample ===")
cleaned_df.head()


In [ ]:
# Create visualizations if numeric columns exist
numeric_cols = cleaned_df.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) > 0:
    # Distribution of numeric columns
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.ravel()
    
    for idx, col in enumerate(numeric_cols[:4]):
        if idx < len(axes):
            cleaned_df[col].hist(bins=50, ax=axes[idx], edgecolor='black')
            axes[idx].set_title(f'Distribution of {col}')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
else:
    print("No numeric columns found for visualization")


## 6. Summary & Next Steps

### Completed:
- ✅ Data loaded and assessed
- ✅ Duplicates removed
- ✅ Missing values handled
- ✅ Bot traffic filtered
- ✅ Timezones normalized
- ✅ Data quality report generated

### Next Steps:
- Run SQL queries from `outputs/sql_queries/analysis_queries.sql` for deeper analysis
- Create Tableau dashboards using the cleaned data
- Implement advanced analytics and predictive models


In [ ]:
# Final summary
print("=== Data Cleaning Pipeline Complete ===")
print(f"Cleaned data saved to: data/processed/cleaned_data.csv")
print(f"Quality report saved to: data/processed/data_quality_report.csv")
print(f"\nFinal Dataset Shape: {cleaned_df.shape}")
print(f"Total Records: {len(cleaned_df):,}")
print(f"Total Columns: {len(cleaned_df.columns)}")
